### Build the training script ###

In [45]:
import os
import numpy as np
import pandas as pd
import logging
import glob
import json
import datetime
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import patches

# PyTorch framework
import torch

# Hugging Face Library
from transformers import RTDetrV2ForObjectDetection, RTDetrImageProcessor
from transformers import TrainingArguments, Trainer

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import detection as dt
from detection.detrdataset import get_gpu_info, DetectionDatasetFromDF
from detection.detransform import DetrTransform
from detection.imageproc import clipxywh, ImageData
from detection.mapeval import MAPEvaluator

print(f'Project module version: {dt.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project module version: 0.0.1.post1.dev26+g0781d45


In [46]:
# Set device
device, device_str = get_gpu_info()

# Save the date in a string
date_str = datetime.date.today().strftime('%y%m%d')
print(f'Date: {date_str}')

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3060 Laptop GPU
PyTorch version:   2.6.0a0+ecf3bae40a.nv25.01
CUDA version:      12.8
CUDNN version:     90700
Date: 250224


### Locations for the data ###

In [32]:
# Data directory
data_root = os.path.join(os.environ.get('HOME'), 'data')
data_dir = os.path.join(data_root, 'dentex_detection')

# Save model checkpoints
model_dir = os.path.join(data_dir, 'model')
Path(model_dir).mkdir(exist_ok=True, parents=True)

# Model name
model_version = 1
model_name = f'rtdetr_{date_str}_{str(model_version).zfill(2)}'
print(f'Training model: {model_name}')

# Log files
log_dir = os.path.join(model_dir, 'log')
Path(log_dir).mkdir(exist_ok=True, parents=True)

# Image directory and annotation data
image_dir = os.path.join(data_dir, 'quadrants')
annotation_file_name = 'train_split_250224.parquet'
annotation_file = os.path.join(image_dir, annotation_file_name)

# Check the images on disk
file_list = glob.glob(os.path.join(image_dir, '*.png'))
expected_n_images = 2531
if not len(file_list) == expected_n_images:
    print(f'WARNING: expected number of images ({expected_n_images}) does not match the number of images on disk.')
    print(f'Delete files and start over.')
else:
    print(f'Found {len(file_list)} images.')

Training model: rtdetr_250224_01
Found 2531 images.


### Load the annotations ###

In [23]:
df = pd.read_parquet(annotation_file)
# Create the label column (the tooth position, but starting from 0)
label_name_list = sorted(list(df['ada'].unique()))
id2label = dict(zip(range(len(label_name_list)), label_name_list))
id2label = {int(label_id): str(name) for label_id, name in id2label.items()}
label2id = {str(name): int(label_id) for label_id, name in id2label.items()}

In [27]:
# Add columns with the file paths and the labels
df = df.assign(file=df['file_name'].apply(lambda f: os.path.join(image_dir, f)),
               label=df['ada'].apply(lambda name: label2id.get(str(name))))
display(df.head())
print(sorted(list(df['label'].unique())))

,file_name,quadrant,pos,bbox,segmentation,fdi,ada,dset,file,label
0,train_0_1.png,1,1,"[666, 102, 103, 376]","[[757, 478, 769, 102, 678, 113, 666, 469]]",11,8,train,/app/data/dentex_detection/quadrants/train_0_1...,7
1,train_0_1.png,1,2,"[593, 107, 85, 377]","[[666, 484, 678, 110, 607, 107, 604, 299, 619,...",12,7,train,/app/data/dentex_detection/quadrants/train_0_1...,6
2,train_0_1.png,1,3,"[531, 69, 85, 368]","[[587, 437, 616, 357, 607, 72, 534, 69, 531, 4...",13,6,train,/app/data/dentex_detection/quadrants/train_0_1...,5
3,train_0_1.png,1,4,"[457, 31, 115, 403]","[[522, 434, 572, 378, 543, 31, 463, 40, 457, 3...",14,5,train,/app/data/dentex_detection/quadrants/train_0_1...,4
4,train_0_1.png,1,5,"[369, 10, 100, 406]","[[437, 416, 469, 378, 466, 10, 381, 31, 378, 2...",15,4,train,/app/data/dentex_detection/quadrants/train_0_1...,3


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [28]:
# Augmentations
use_transform = 'transform_1'

# Hugging face model checkpoint
hf_checkpoint = 'PekingU/rtdetr_v2_r101vd'
image_processor = RTDetrImageProcessor.from_pretrained(hf_checkpoint)
model = RTDetrV2ForObjectDetection.from_pretrained(hf_checkpoint,
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   anchor_image_size=None,
                                                   ignore_mismatched_sizes=True)

# Custom collate_fn to batch the images
def collate_fn(batch):
    data = {}
    data["pixel_values"] = torch.stack([x["pixel_values"] for x in batch])
    data["labels"] = [x["labels"] for x in batch]
    return data

Some weights of RTDetrV2ForObjectDetection were not initialized from the model checkpoint at PekingU/rtdetr_v2_r101vd and are newly initialized because the shapes did not match:
- model.decoder.class_embed.0.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([32]) in the model instantiated
- model.decoder.class_embed.0.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([32, 256]) in the model instantiated
- model.decoder.class_embed.1.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([32]) in the model instantiated
- model.decoder.class_embed.1.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([32, 256]) in the model instantiated
- model.decoder.class_embed.2.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([32]) in the model instantiated
- model.decoder.class_embed.2.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([32, 256]) in the model instantiated
- mode

### Set up logging ###

In [38]:
log_file_name = f'train_log_{date_str}.log'
log_file = os.path.join(log_dir, log_file_name)
dtfmt = '%y%m%d-%H:%M'
logfmt = '%(asctime)s-%(name)s-%(levelname)s-%(message)s'
logging.basicConfig(filename=log_file,
                    filemode='w',
                    level=logging.INFO,
                    format=logfmt,
                    datefmt=dtfmt)
logger = logging.getLogger(name=__name__)

### Training arguments ###

In [40]:
training_args_dict = {'output_dir': os.path.join(model_dir, model_name), 
                      'num_train_epochs': 20, 
                      'max_grad_norm': 0.1, 
                      'learning_rate': 5e-5, 
                      'warmup_steps': 300,
                      'per_device_train_batch_size': 4, 
                      'dataloader_num_workers': 2, 
                      'metric_for_best_model': 'eval_map',
                      'greater_is_better': True, 
                      'load_best_model_at_end': True,
                      'eval_strategy': 'epoch', 
                      'save_strategy': 'epoch', 
                      'save_total_limit': 2, 
                      'remove_unused_columns': False,
                      'eval_do_concat_batches': False}

logger.info(json.dumps(training_args_dict))

### Create the data sets ###

In [30]:
# Create the image transformations for the data sets
train_transform = DetrTransform(use_transform).train_transform()
val_transform = DetrTransform(use_transform).val_transform()

# Create the data sets
dset_list = sorted(list(df['dset'].unique()))
print(dset_list)
dataset_dict = {}
for dset in dset_list:
    df_dset = df.loc[df['dset'] == dset]
    if dset == 'train':
        transform = train_transform
    else:
        transform = val_transform
    dataset = DetectionDatasetFromDF(data=df_dset, 
                                     processor=image_processor, 
                                     file_col='file', 
                                     label_col='label', 
                                     bbox_col='bbox', 
                                     transform=transform, 
                                     bbox_format='xywh', 
                                     validate=True)
    dataset_dict.update({dset: dataset})
    print(f'Number of images in {dset.upper()}: {len(dataset)}')

['test', 'train', 'val']
Number of images in TEST: 32
Number of images in TRAIN: 2479
Number of images in VAL: 20


In [49]:
# Set the evaluation metrics
eval_compute_metrics_fn = MAPEvaluator(image_processor=image_processor, threshold=0.01, id2label=id2label)
training_args = TrainingArguments(**training_args_dict)

# Set up the training
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=dataset_dict.get('train'),
                  eval_dataset=dataset_dict.get('val'),
                  processing_class=image_processor,
                  data_collator=collate_fn,
                  compute_metrics=eval_compute_metrics_fn)

# Run the training
trainer.train()

In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss,Map,Map 50,Map 75,Map Small,Map Medium,Map Large,Mar 1,Mar 10,Mar 100,Mar Small,Mar Medium,Mar Large,Map 1,Mar 100 1,Map 2,Mar 100 2,Map 3,Mar 100 3,Map 4,Mar 100 4,Map 5,Mar 100 5,Map 6,Mar 100 6,Map 7,Mar 100 7,Map 8,Mar 100 8,Map 9,Mar 100 9,Map 10,Mar 100 10,Map 11,Mar 100 11,Map 12,Mar 100 12,Map 13,Mar 100 13,Map 14,Mar 100 14,Map 15,Mar 100 15,Map 16,Mar 100 16,Map 17,Mar 100 17,Map 18,Mar 100 18,Map 19,Mar 100 19,Map 20,Mar 100 20,Map 21,Mar 100 21,Map 22,Mar 100 22,Map 23,Mar 100 23,Map 24,Mar 100 24,Map 25,Mar 100 25,Map 26,Mar 100 26,Map 27,Mar 100 27,Map 28,Mar 100 28,Map 29,Mar 100 29,Map 30,Mar 100 30,Map 31,Mar 100 31,Map 32,Mar 100 32
1,38.122400,14.336624,0.386100,0.668700,0.410500,-1.000000,0.000000,0.389900,0.505000,0.687800,0.726100,-1.000000,0.000000,0.732900,0.303300,0.800000,0.484800,0.800000,0.236400,0.833300,0.453300,0.850000,0.063400,0.675000,0.478100,0.760000,0.394800,0.720000,0.653000,0.840000,0.449400,0.760000,0.531800,0.680000,0.477500,0.800000,0.202100,0.660000,0.367300,0.650000,0.372600,0.833300,0.519200,0.780000,0.250200,0.800000,0.250000,0.733300,0.110400,0.700000,0.327300,0.500000,0.049700,0.766700,0.335100,0.800000,0.505000,0.700000,0.418200,0.560000,0.223300,0.740000,0.214900,0.720000,0.450400,0.540000,0.355400,0.560000,0.488400,0.700000,0.530600,0.700000,0.662400,0.760000,0.542800,0.740000,0.653200,0.775000
2,23.280000,13.048721,0.495000,0.780000,0.523300,-1.000000,0.000000,0.500600,0.581100,0.689800,0.722400,-1.000000,0.000000,0.727700,0.322200,0.800000,0.402300,0.800000,0.600300,0.866700,0.215000,0.850000,0.702900,0.725000,0.632000,0.740000,0.462600,0.760000,0.643100,0.840000,0.461300,0.760000,0.474200,0.600000,0.515600,0.780000,0.437100,0.780000,0.410100,0.500000,0.599400,0.700000,0.428300,0.740000,0.750500,0.750000,0.273800,0.633300,0.185100,0.633300,0.200400,0.600000,0.403100,0.700000,0.728300,0.860000,0.697900,0.780000,0.465500,0.560000,0.094900,0.560000,0.546900,0.800000,0.486800,0.640000,0.434700,0.620000,0.557700,0.625000,0.676100,0.820000,0.703100,0.800000,0.615500,0.720000,0.712700,0.775000


NOTE! Installing ujson may make loading annotations faster.


KeyboardInterrupt: 